In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
print('Tensorflow version: ', tf.__version__)

Tensorflow version:  2.15.0


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_csv(r"/content/drive/MyDrive/BERT Variations/all_in_one.csv", encoding='utf-8')
df = df.fillna(method='ffill')
df.head(20)

,Sentence ID,Word,Tag
0,1,The,O
1,1,admin@338,B-HackOrg
2,1,has,O
3,1,largely,O
4,1,targeted,O
5,1,organizations,O
6,1,involved,O
7,1,in,O
8,1,financial,B-Idus
9,1,",",O


In [6]:
df.groupby('Tag').size().reset_index(name='counts')

,Tag,counts
0,B-Area,2531
1,B-Exp,1331
2,B-Features,1025
3,B-HackOrg,4214
4,B-Idus,1579
5,B-OffAct,1662
6,B-Org,1361
7,B-Purp,918
8,B-SamFile,1660
9,B-SecTeam,1327


In [7]:
print('Unique words in corpus: ', df['Word'].nunique())
print('Unique tags in corpus: ', df['Tag'].nunique())

Unique words in corpus:  9529
Unique tags in corpus:  27


In [8]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df['Tag'])
df['Enc_tag'] = le.transform(df['Tag'])

In [11]:
class SentenceGetter(object):
    def __init__(self, df):
        self.n_sent = 1
        self.df = df
        agg_func = lambda s: [w for w in s['Word'].values.tolist()]
        self.grouped = self.df.groupby('Sentence ID').apply(agg_func)
        self.sentences = [s for s in self.grouped]

getter = SentenceGetter(df)
sentences = getter.sentences

In [13]:
class TagGetter(object):
    def __init__(self, df):
        self.n_sent = 1
        self.df = df
        agg_func = lambda s: [w for w in s['Enc_tag'].values.tolist()]
        self.grouped = self.df.groupby('Sentence ID').apply(agg_func)
        self.sentences = [s for s in self.grouped]

getter = TagGetter(df)
Tags= getter.sentences

In [14]:
data = {'Sentence ID' : df['Sentence ID'].unique(),'Word': sentences,'Tag': Tags }
df1 = pd.DataFrame(data = data)

In [15]:
df1

,Sentence ID,Word,Tag
0,1,"[The, admin@338, has, largely, targeted, organ...","[26, 3, 26, 26, 26, 26, 26, 26, 4, 26, 4, 26, ..."
1,2,"[The, admin@338, started, targeting, Hong, Kon...","[26, 3, 26, 26, 0, 13, 6, 19, 26, 26, 26, 26, ..."
2,3,"[Multiple, China-based, cyber, threat, groups,...","[26, 0, 3, 16, 16, 26, 26, 6, 19, 19, 26, 26, ..."
3,4,"[The, admin@338, has, targeted, international,...","[26, 3, 26, 26, 6, 19, 19, 26, 26, 26, 26]"
4,5,"[In, August, 2015, ,, the, admin@338, sent, sp...","[26, 10, 23, 26, 26, 3, 26, 12, 25, 25, 26, 26..."
...,...,...,...
6577,6588,"[APT33, often, conducts, spear-phishing, opera...","[3, 26, 26, 12, 26, 26, 26, 26, 12, 25, 26]"
6578,6589,"[In, a, recent, attack, ,, APT33, sent, spear-...","[26, 26, 26, 5, 26, 3, 26, 12, 25, 26, 26, 26,..."
6579,6590,"[These, emails, included, recruitment-themed, ...","[26, 12, 26, 12, 25, 26, 26, 26, 26, 11, 24, 2..."
6580,6591,"[APT34, often, uses, compromised, accounts, to...","[3, 26, 26, 11, 24, 26, 26, 12, 26, 26]"


In [16]:
from sklearn.model_selection import train_test_split

training_dataset, testing_dataset = train_test_split(df1, test_size=0.2, random_state=2018)

In [17]:
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 8.5 MB/s eta 0:00:00


In [18]:
import pandas as pd
from datasets import Dataset

def dataframe_to_conll(df):
    conll_lines = []
    for i, row in df.iterrows():
        word = row['Word']
        #pos = row['POS']
        ner = row['Tag']
        conll_lines.append(f"{word}{ner}")

    conll_dataset = '\n'.join(conll_lines)
    return conll_dataset

conll_dataset = dataframe_to_conll(training_dataset)
dataset = Dataset.from_pandas(training_dataset)

In [19]:
raw_tags = df.Tag.unique().tolist()
print(raw_tags)

['O', 'B-HackOrg', 'B-Idus', 'I-Idus', 'B-Tool', 'I-Tool', 'B-Area', 'I-Area', 'B-Org', 'I-Org', 'I-HackOrg', 'B-Time', 'I-Time', 'B-Way', 'I-Way', 'B-OffAct', 'B-SamFile', 'B-Features', 'I-Features', 'I-OffAct', 'B-SecTeam', 'B-Exp', 'I-Exp', 'I-SecTeam', 'B-Purp', 'I-Purp', 'I-SamFile']


In [20]:
tags = ['<PAD>'] + raw_tags
print(tags)

['<PAD>', 'O', 'B-HackOrg', 'B-Idus', 'I-Idus', 'B-Tool', 'I-Tool', 'B-Area', 'I-Area', 'B-Org', 'I-Org', 'I-HackOrg', 'B-Time', 'I-Time', 'B-Way', 'I-Way', 'B-OffAct', 'B-SamFile', 'B-Features', 'I-Features', 'I-OffAct', 'B-SecTeam', 'B-Exp', 'I-Exp', 'I-SecTeam', 'B-Purp', 'I-Purp', 'I-SamFile']


In [21]:
from sklearn import preprocessing
le2 = preprocessing.LabelEncoder()
le2.fit(tags)

LabelEncoder()

In [22]:
TAG_SIZE = len(tags)
VOCAB_SIZE = 20000


import matplotlib.pyplot as plt
import copy

import numpy as np
import tensorflow as tf


train_tokens = tf.ragged.constant(dataset['Word'])
train_tokens = tf.map_fn(tf.strings.lower, train_tokens)

lookup_layer = tf.keras.layers.StringLookup(max_tokens=VOCAB_SIZE, mask_token="[MASK]", oov_token="[UNK]")
lookup_layer.adapt(train_tokens)

print(len(lookup_layer.get_vocabulary()))
print(lookup_layer.get_vocabulary()[:10])


def create_data_generator(dataset):
  def data_generator():
    for item in dataset:
      yield item['Word'], item['Tag']

  return data_generator

data_signature= (
        tf.TensorSpec(shape=(None,), dtype=tf.string),
        tf.TensorSpec(shape=(None, ), dtype=tf.int32)
)

train_data = tf.data.Dataset.from_generator(
    create_data_generator(dataset),
    output_signature=data_signature
)


def dataset_preprocess(tokens, tag_ids):
    preprocessed_tokens = preprecess_tokens(tokens)

    # increase by 1 for all tag_ids,
    # because `<PAD>` is added as the first element in tags list
    preprocessed_tag_ids = tag_ids + 1

    return preprocessed_tokens, preprocessed_tag_ids

def preprecess_tokens(tokens):
    tokens = tf.strings.lower(tokens)
    return lookup_layer(tokens)

BATCH_SIZE = 128

train_dataset = (
    train_data.map(dataset_preprocess)
    .padded_batch(batch_size=BATCH_SIZE).cache()
)




8133
['[MASK]', '[UNK]', 'the', ',', '.', 'to', 'and', 'in', 'a', 'of']


In [23]:
def build_embedding_bilstm_model(
    vocab_size: int, embed_dims: int, lstm_units: int, tag_size: int
) -> tf.keras.Model:
    x = tf.keras.layers.Input(shape=(None,), dtype=tf.int64, name="x")
    y = tf.keras.layers.Embedding(vocab_size, embed_dims, mask_zero=True)(x)
    y = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(lstm_units, return_sequences=True)
    )(y)
    output = tf.keras.layers.Dense(tag_size, activation='softmax')(y)

    return tf.keras.Model(inputs=x, outputs=output)


model = build_embedding_bilstm_model(VOCAB_SIZE, 64, 128, TAG_SIZE)

In [24]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [25]:
train_dataset

EPOCHS = 40
BATCH_SIZE = 32

history = model.fit(train_dataset, epochs=EPOCHS, batch_size=BATCH_SIZE)



Epoch 1/40
42/42 [==============================] - 35s 531ms/step - loss: 1.8100 - accuracy: 0.7745
Epoch 2/40
42/42 [==============================] - 20s 466ms/step - loss: 1.1982 - accuracy: 0.7928
Epoch 3/40
42/42 [==============================] - 21s 487ms/step - loss: 1.0738 - accuracy: 0.7928
Epoch 4/40
42/42 [==============================] - 18s 433ms/step - loss: 0.8998 - accuracy: 0.7939
Epoch 5/40
42/42 [==============================] - 19s 464ms/step - loss: 0.8184 - accuracy: 0.8004
Epoch 6/40
42/42 [==============================] - 23s 556ms/step - loss: 0.7645 - accuracy: 0.8086
Epoch 7/40
42/42 [==============================] - 21s 481ms/step - loss: 0.7041 - accuracy: 0.8189
Epoch 8/40
42/42 [==============================] - 21s 505ms/step - loss: 0.6371 - accuracy: 0.8307
Epoch 9/40
42/42 [==============================] - 18s 438ms/step - loss: 0.5647 - accuracy: 0.8470
Epoch 10/40
42/42 [==============================] - 19s 441ms/step - loss: 0.4816 - accura

In [26]:
test_conll_dataset = dataframe_to_conll(testing_dataset)
test_dataset = Dataset.from_pandas(testing_dataset)

def create_data_generator(dataset):
  def data_generator():
    for item in dataset:
      yield item['Word'], item['Tag']

  return data_generator

data_signature= (
        tf.TensorSpec(shape=(None,), dtype=tf.string),
        tf.TensorSpec(shape=(None, ), dtype=tf.int32)
)

test_data = tf.data.Dataset.from_generator(
    create_data_generator(test_dataset),
    output_signature=data_signature
)

test_dataset = (
    test_data.map(dataset_preprocess)
    .padded_batch(batch_size=BATCH_SIZE).cache()
)

In [27]:
model.evaluate(test_dataset)

42/42 [==============================] - 8s 63ms/step - loss: 0.3594 - accuracy: 0.9302


[0.35942840576171875, 0.9302495718002319]

In [28]:
import numpy as np
predicted_tags_list = []
true_tags_list = []
for i in range(len(testing_dataset)):
    Test_case = testing_dataset.iloc[i]['Word']
    true_tags_list.extend(le.inverse_transform(testing_dataset.iloc[i]['Tag']))
    # Preprocess the test sentence (similar to what you did during training)
    preprocessed_test_sentence = preprecess_tokens(Test_case)

    # Reshape the preprocessed input to match the model's input shape
    input_sequence = np.array(preprocessed_test_sentence)
    input_sequence = np.expand_dims(input_sequence, axis=0)

    # Predict the tags for the test sentence
    predictions = model.predict(input_sequence)

    # Decode the predictions to obtain the predicted tags
    predicted_tags = np.argmax(predictions, axis=-1)[0]

    # Inverse transform the predicted tags to get the original labels
    predicted_tags_list.extend(list(le2.inverse_transform(predicted_tags)))

1/1 [==============================] - 0s 45ms/step


In [29]:
p = []
t = []
for i in predicted_tags_list:
  if i == 'O':
    p.append(i)
  else:
    p.append(i[2:])

for i in true_tags_list:
  if i == 'O':
    t.append(i)
  else:
    t.append(i[2:])

In [31]:
from sklearn.metrics import classification_report
print(classification_report(t, p))

report = classification_report(t, p, digits=4, output_dict=True)
f1_weighted = report['weighted avg']['f1-score']

recall_weighted = report['weighted avg']['recall']
precision_weighted = report['weighted avg']['precision']

# Print the results

print ('Weighted F1 Score: ', f1_weighted)
print ('Weighted Recall: ', recall_weighted)
print ('Weighted Precision: ', precision_weighted)

              precision    recall  f1-score   support

        Area       0.88      0.91      0.90       728
         Exp       0.96      0.96      0.96       378
    Features       0.91      0.93      0.92       563
     HackOrg       0.78      0.86      0.82      1168
        Idus       0.80      0.82      0.81       531
           O       0.97      0.96      0.96     27798
      OffAct       0.69      0.81      0.75       567
         Org       0.80      0.72      0.76       593
        Purp       0.78      0.83      0.80       525
     SamFile       0.86      0.92      0.89       423
     SecTeam       0.88      0.87      0.88       388
        Time       0.92      0.91      0.92       571
        Tool       0.76      0.75      0.75       955
         Way       0.69      0.78      0.73       396

    accuracy                           0.93     35584
   macro avg       0.83      0.86      0.85     35584
weighted avg       0.94      0.93      0.93     35584

Weighted F1 Score:  0.934

In [32]:
report = classification_report(t, p, digits=4, output_dict=True)
# Access the weighted F1 score, recall, and precision
f1_weighted = report['macro avg']['f1-score']

recall_weighted = report['macro avg']['recall']
precision_weighted = report['macro avg']['precision']

# Print the results

print ('Macro F1 Score: ', f1_weighted)
print ('Macro Recall: ', recall_weighted)
print ('Macro Precision: ', precision_weighted)

Macro F1 Score:  0.8454826571334094
Macro Recall:  0.8583641293756153
Macro Precision:  0.8346810601555645


In [33]:
import numpy as np

# Sample test sentence
test_sentence = "Google has agreed to pay $93 million to settle a lawsuit filed by the U.S. state of California over allegations that the company's location-privacy practices misled consumers and violated consumer protection laws."

preprocessed_test_sentence = preprecess_tokens(test_sentence.split())

# Reshape the preprocessed input to match the model's input shape
input_sequence = np.array(preprocessed_test_sentence)
input_sequence = np.expand_dims(input_sequence, axis=0)

predictions = model.predict(input_sequence)

# Decode the predictions to obtain the predicted tags
predicted_tags = np.argmax(predictions, axis=-1)[0]

# Inverse transform the predicted tags to get the original labels
predicted_tags = list(le2.inverse_transform(predicted_tags))

for token, label in zip(test_sentence.split(), predicted_tags):
    print("{:20}\t{}".format(token, label))


1/1 [==============================] - 0s 28ms/step
Google              	B-SecTeam
has                 	O
agreed              	O
to                  	O
pay                 	O
$93                 	O
million             	O
to                  	O
settle              	O
a                   	O
lawsuit             	O
filed               	O
by                  	O
the                 	O
U.S.                	B-Area
state               	O
of                  	O
California          	B-Area
over                	O
allegations         	O
that                	O
the                 	O
company's           	O
location-privacy    	I-Area
practices           	I-Area
misled              	O
consumers           	O
and                 	O
violated            	O
consumer            	B-Org
protection          	I-Org
laws.               	I-Org
